In [ ]:
import keras
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, confusion_matrix
import os
import seaborn as sns
import sys
from datetime import datetime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.plotting import plot_confusion_matrix
from model.trainer import Cifar10Trainer
from data.processing import standardize_data
from model.evaluate import evaluate_model
from utils.plotting import plot_confusion_matrix, plot_multi_auc

%matplotlib inline

## Load Dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
label_list = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
label_decoder = {i: label for i, label in enumerate(label_list)}
print(f"the training dataset image dimensions are: {x_train.shape}")
print(f"the training dataset label dimensions are: {y_train.shape}")
print(label_decoder)

In [ ]:
label_collections = {}
for i in range(10):
    indices, _ = np.where(y_train == i)
    label_collections[i] = indices
    print(f"number of {label_decoder[i]} examples: {indices.size}")

## Precision / Recall

In [ ]:
fig = plt.figure(figsize=(10, 10))
columns = 4
rows = 5
proba = 0.8
true_labels = []
preds = []
for i in range(1, columns*rows +1):
    img_id = np.random.choice(x_train.shape[0])
    img = x_train[img_id]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.xticks([], [])
    plt.yticks([], [])
    true_labels.append(y_train[img_id].item())
    if np.random.rand(1) > proba:
        preds.append(np.random.choice([x for x in range(10) if x != true_labels[i-1]]))
    else:
        preds.append(true_labels[i-1])
    plt.title(f"actual: {label_decoder[true_labels[i-1]]} \n predicted: {label_decoder[preds[i-1]]}")
fig.tight_layout()
plt.show()

## Confusion Matrix

In [ ]:
precision = precision_score(true_labels, preds, average="micro")
print(f"precision score of the model is {precision}")
plot_confusion_matrix(true_labels, preds, classes=label_list,
                      title='Confusion matrix, without normalization')

## RGB Image

In [ ]:
image = x_train[43]
fig = plt.figure(figsize=(10, 4))
fig.add_subplot(1, 4, 1)
plt.imshow(image)
plt.title("Colored")
titles = ["R", "G", "B"]
for i in range(3):
    fig.add_subplot(1, 4, i + 2)
    plt.imshow(image[:,:,i], cmap='gray')
    plt.title(titles[i])

## Label one-hot-encoding

In [ ]:
print(y_train[:10, 0])
print([label_decoder[x] for x in y_train[:10, 0]])

In [ ]:
y_train_one_hot = keras.utils.to_categorical(y_train, num_classes=np.unique(y_train).size)
print(y_train_one_hot[:10,:])

In [ ]:
one_hot_plot = y_train_one_hot[:10,:].astype(int)
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(one_hot_plot, interpolation='nearest', cmap="Blues")
fmt = 'd'
thresh = np.max(one_hot_plot) / 2
for i in range(one_hot_plot.shape[0]):
    for j in range(one_hot_plot.shape[1]):
        ax.text(j, i, format(one_hot_plot[i, j], fmt),
                ha="center", va="center",
                color="white" if one_hot_plot[i, j] > thresh else "black")
fig.tight_layout()
plt.xlabel("label")
plt.ylabel("example")
plt.show()

## Data Preprocessing

Original pixel value distribution

In [ ]:
test_images = x_train[:100].copy()
sns.distplot(test_images.ravel(), bins=50)
plt.show()

Mean subtraction

In [ ]:
test_images = test_images.astype(float)
test_images -= np.mean(test_images)
sns.distplot(test_images.ravel(), bins=50)
plt.show()

Standardization

In [ ]:
test_images /= np.std(test_images)
sns.distplot(test_images.ravel(), bins=50)
plt.show()

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)
x_train = standardize_data(x_train)
x_test = standardize_data(x_test)

## Model Training

In [ ]:
artifact_base = "/tmp/artifact/cifar10"

hyperparameters = {
  "learning_rate": 0.001,
  "batch_normalization": True,
  "weight_decay": 1e-4,
  "base_filters": 32,
  "batch_size": 64,
  "fc_size": 128,
  "dropout": 0.2,
  "lr_decay": 1e-6,
  "rotation_angle": 15,
  "width_shift_range": 0.1, 
  "height_shift_range": 0.1,
  "shear_range": 0.1,
  "zoom_range": 0.1,
  "horizontal_flip": True,
  "early_stopping_patience": 15,
  "reduce_lr_patience": 6,
  "reduce_lr_factor": 0.3,
  "activation": "relu",
  "classifier_activation": "softmax",
  "loss": "categorical_crossentropy"
}
job_name = "training_test_"
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
artifact_directory = os.path.join(artifact_base, job_name+timestamp)

In [ ]:
trainer = Cifar10Trainer(
    model_name="convnet6",
    hyperparameters=hyperparameters,
    artifact_directory=artifact_directory,
    x_train=x_train,
    y_train=y_train,
    x_test=x_test,
    y_test=y_test
)

In [ ]:
trainer.train()

## Evaluate model

In [ ]:
model_artifact = "../artifacts/weights.92-0.449348.hdf5"

In [ ]:
fpr, tpr, roc_auc, cm = evaluate_model(model_artifact, x_test, y_test, label_list)